In [54]:
import numpy as np
import pandas as pd
from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests
from itertools import combinations

Классификатор C4.5 и три его модификации: с оптимизацией гиперпараметра m, гиперпараметра cf и с одновременной оптимизацией обоих гиперпараметров. Эти четыре классификатора сравнивались на 14 наборах данных. На каждом датасете был посчитан AUC каждого классификатора. 
Используя критерий знаковых рангов, проведите попарное сравнение каждого классификатора с каждым. Выберите два классификатора, различие между которыми наиболее статистически значимо.

In [2]:
auc = pd.read_csv('AUCs.txt', sep='\t')

In [20]:
auc

,Unnamed: 0,C4.5,C4.5+m,C4.5+cf,C4.5+m+cf
0,adult (sample),0.763,0.768,0.771,0.798
1,breast cancer,0.599,0.591,0.590,0.569
2,breast cancer wisconsin,0.954,0.971,0.968,0.967
3,cmc,0.628,0.661,0.654,0.657
4,ionosphere,0.882,0.888,0.886,0.898
5,iris,0.936,0.931,0.916,0.931
6,liver disorders,0.661,0.668,0.609,0.685
7,lung cancer,0.583,0.583,0.563,0.625
8,lymphography,0.775,0.838,0.866,0.875
9,mushroom,1.000,1.000,1.000,1.000


In [24]:
z = pd.DataFrame(columns=['clf1', 'clf2', 'stat', 'p-val'])

In [17]:
combinations([0,1,2,3], 2)

In [30]:
n = 0
for i, j in combinations(auc.columns[1:], 2):
    s, p = stats.wilcoxon(auc[i], auc[j])
    z.loc[n] = [i, j, s, p]
    n+=1

In [45]:
z

,clf1,clf2,stat,p-val
0,C4.5,C4.5+m,6.5,0.010757
1,C4.5,C4.5+cf,43.0,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
5,C4.5+cf,C4.5+m+cf,10.0,0.022909


In [43]:
z[z['p-val'] == z['p-val'].min()][['clf1', 'clf2']]

,clf1,clf2
0,C4.5,C4.5+m


Сколько статистически значимых на уровне 0.05 различий мы обнаружили?

In [49]:
z[z['p-val'] <= 0.05].shape[0]

4

Судя по данным из предыдущего опроса, настройка какого из параметров классификатора даёт более значимое увеличение качества?


Сравнивая 4 классификатора между собой, мы проверили 6 гипотез. Давайте сделаем поправку на множественную проверку. Начнём с метода Холма. Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки этим методом?

In [56]:
z['reject'], z['p_correct'], _, _ =  multipletests(z['p-val'], alpha = 0.05, method = 'holm')

In [57]:
z

,clf1,clf2,stat,p-val,reject,p_correct
0,C4.5,C4.5+m,6.5,0.010757,False,0.064543
1,C4.5,C4.5+cf,43.0,0.861262,False,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906,False,0.079532
3,C4.5+m,C4.5+cf,17.0,0.046333,False,0.138998
4,C4.5+m,C4.5+m+cf,22.0,0.327826,False,0.655651
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,False,0.091636


Сколько гипотез можно отвергнуть на уровне значимости 0.05 после поправки методом Бенджамини-Хохберга?

In [58]:
z['reject'], z['p_correct'], _, _ =  multipletests(z['p-val'], alpha = 0.05, method = 'fdr_bh')

In [59]:
z

,clf1,clf2,stat,p-val,reject,p_correct
0,C4.5,C4.5+m,6.5,0.010757,True,0.045818
1,C4.5,C4.5+cf,43.0,0.861262,False,0.861262
2,C4.5,C4.5+m+cf,11.0,0.015906,True,0.045818
3,C4.5+m,C4.5+cf,17.0,0.046333,False,0.069499
4,C4.5+m,C4.5+m+cf,22.0,0.327826,False,0.393391
5,C4.5+cf,C4.5+m+cf,10.0,0.022909,True,0.045818


In [33]:
z.sort_values(by=['p-val'])

,clf1,clf2,stat,p-val
0,C4.5,C4.5+m,6.5,0.010757
2,C4.5,C4.5+m+cf,11.0,0.015906
5,C4.5+cf,C4.5+m+cf,10.0,0.022909
3,C4.5+m,C4.5+cf,17.0,0.046333
4,C4.5+m,C4.5+m+cf,22.0,0.327826
1,C4.5,C4.5+cf,43.0,0.861262


In [35]:
0.015906*(6-2+1)

0.07953

In [37]:
0.05/6

0.008333333333333333